# 모듈 불러오기

#### 기본

In [ ]:
import numpy as np
import pandas as pd
import joblib

In [ ]:
# !conda install numpy 
# !conda install pandas
# !conda install scikit-learn
# !conda install scipy
# !conda install tensorflow
# !conda install matplotlib
# !conda install seaborn

# !pip install xgboost
# !pip install lightgbm
# !pip install catboost
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !pip install hyperopt
# !pip install -U imbalanced-learn
# !pip install missingno

#### 전처리

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

from sklearn import impute
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer

#### 리샘플링

In [ ]:
from imblearn.over_sampling import (
    RandomOverSampler, 
    ADASYN, 
    SMOTE
)
from imblearn.under_sampling import (
    RandomUnderSampler, 
    TomekLinks, 
    CondensedNearestNeighbour, 
    OneSidedSelection, 
    EditedNearestNeighbours, 
    NeighbourhoodCleaningRule
)

#### 분석

In [ ]:
from scipy.stats import skew, kurtosis
from scipy.stats import ttest_ind, f_oneway, pearsonr, chi2_gen

#### 회귀

In [ ]:
from sklearn.linear_model import LinearRegression as RL
from sklearn.neighbors import KNeighborsRegressor as KNNR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR
from xgboost import XGBRegressor as XGBR
from lightgbm import LGBMRegressor as LGBMR
from catboost import CatBoostRegressor as CBR

from lightgbm import plot_importance as lgbm_plot_importance
from xgboost import plot_importance as xgb_plot_importance
from catboost import Pool

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

#### 분류

In [ ]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from xgboost import XGBClassifier as XGBC
from lightgbm import LGBMClassifier as LGBMC
from catboost import CatBoostClassifier as CBC

from sklearn.metrics import confusion_matrix as cmatrix
from sklearn.metrics import classification_report as creport
from sklearn.metrics import recall_score as recall
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

#### 교차검증

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import cv
import hyperopt

from sklearn.model_selection import (
    StratifiedKFold, # 분류
    KFold, # 회귀
    # GroupKFold, 
    # RepeatedKFold, 
    # StratifiedGroupKFold, 
    # RepeatedStratifiedKFold
)

#### 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rc('font', family='Malgun Gothic')
sns.set(font="Malgun Gothic",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')  

import warnings
warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

# 데이터 로드

In [ ]:
folder_name = '1th'

x_train = pd.read_csv(f'./{folder_name}/x_train.csv', sep=',', encoding='utf-8')
y_train = pd.read_csv(f'./{folder_name}/y_train.csv', sep=',', encoding='utf-8')
x_val = pd.read_csv(f'./{folder_name}/x_test.csv', sep=',', encoding='utf-8')
y_val = pd.read_csv(f'./{folder_name}/y_test.csv', sep=',', encoding='utf-8')
x_test = pd.read_csv(f'./{folder_name}/test_x.csv', sep=',', encoding='utf-8')

# x_test 전처리

In [ ]:
x_test.drop('id', axis=1, inplace=True)

In [ ]:
knn_imputer = KNNImputer(n_neighbors=5)
x_test = knn_imputer.fit_transform(x_test)
x_test = pd.DataFrame(x_test, columns=x_train.columns)
x_test.info()

# LightGBM 세팅

In [ ]:
model = LGBMC(learning_rate=0.1,)
evals = [(x_val, y_val)]

model.fit(  
            x_train, 
            y_train,
            early_stopping_rounds=100, 
            eval_metric="logloss", 
            eval_set=evals, 
            verbose=True
         )

y_pred = model.predict(x_test)
pred_proba = model.predict_proba(x_test)[:, 1]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 12))

# importance_type 에 따라 다른식으로 중요도를 도출 시킴
# 'split' | 'gain' | 'cover' 
lgbm_plot_importance(model, title='LGBM_Feature_Importance', importance_type='split', ax=ax) 

# Catboost 세팅

- Sparse 한 Matrix 는 처리하지 못한다.
- 데이터 대부분이 수치형 변수인 경우, Light GBM 보다 학습 속도가 느리다. 

#### 회귀

In [ ]:
# https://catboost.ai/en/docs/concepts/python-reference_catboostregressor
# train_pool = Pool(x_train, 
#                   y_train, 
#                   cat_features=[0,2,5])

# test_pool = Pool(x_test, 
#                  cat_features=[0,2,5]) 

model = CBR(iterations=10, 
            # depth=2, 
            # learning_rate=1, 
            loss_function='RMSE',
            random_seed=1,
            task_type="GPU",
            verbose=True)

model.fit(
    x_train, y_train,
    # cat_features=cat_features,
    # eval_set=eval_set,
    early_stopping_rounds=100, 
    verbose=True)

y_pred = model.predict(x_val)

In [ ]:
print('CatBoostRegressor')
print('mse : ', np.round(mse(y_val, y_pred), 6))
print('r2_score : ', np.round(r2_score(y_val, y_pred)))

#### 분류

In [ ]:
# https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier
# test_data = catboost_pool = Pool(x_test, y_test)

eval_set = Pool(x_val,
                y_val,
                ) 

model = CBC(iterations=100,
            # depth=2, 
            # learning_rate=1, 
            loss_function='Logloss',
            random_seed=1,
            task_type="GPU",
            verbose=True)

model.fit(
    x_train, y_train,
    # cat_features=cat_features,
    # eval_set=eval_set,
    plot=True
)

y_pred = model.predict(x_val)
y_pred_proba = model.predict_proba(x_val)

In [ ]:
print(creport(y_val, y_pred))

#### model.predict type 옵션

In [ ]:
preds_class = model.predict(x_test, prediction_type='Class')
preds_proba = model.predict(x_test, prediction_type='Probability')
preds_raw_vals = model.predict(x_test, prediction_type='RawFormulaVal')